### Basic RAG retriever using Langchain and Anthropic model

../notebooks/data/cleaned20242108.csv


In [1]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_anthropic import ChatAnthropic
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]= os.getenv("LANGCHAIN_API_KEY")
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")

loader = WebBaseLoader(
    web_paths=("https://www.gutenberg.org/cache/epub/1399/pg1399-images.html",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("chapter", "pg-boilerplate pgheader", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [11]:
from langchain_chroma import Chroma

In [12]:
persist_directory='db'  

In [ ]:
# vectorstore = Chroma.from_documents(documents=splits, embedding=OllamaEmbeddings(model='nomic-embed-text'),persist_directory=persist_directory)

In [13]:
vectorestore= Chroma(persist_directory=persist_directory,embedding_function=OllamaEmbeddings(model='nomic-embed-text'),)

In [14]:
query="what is the first name of oblonsky?"

In [16]:

retriever = vectorestore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("who is madame karenina")

'Based on the provided context, Madame Karenina appears to be a character in the story who is the subject of gossip and social controversy. She is mentioned as being seen with Alexey Vronsky and is involved in some kind of social incident with Madame Kartasova at what seems to be a theater or opera. However, the context does not provide enough information to definitively state who Madame Karenina is or her full role in the story.'